In [1]:
import pandas as pd

data = pd.read_csv('data/car-sales.csv').drop(columns=["Unnamed: 0"], axis=1)
data.head()

,price,sold,models_age,km_per_year
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
# "Bad" scenario where the classes are ordered in ascending order

bad_data = data.sort_values("sold", ascending=True)
bad_x = bad_data[["price", "models_age","km_per_year"]]
bad_y = bad_data["sold"]
bad_data.head()

,price,sold,models_age,km_per_year
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [3]:
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier
import numpy as np

SEED = 301
np.random.seed(SEED)

model = DummyClassifier()
results = cross_validate(model, bad_x, bad_y, cv = 10, return_train_score=False)
mean = results['test_score'].mean()
std = results['test_score'].std()
print("Accuracy with stratified dummy, 10 = [%.2f, %.2f]" % ((mean - 2 * std)*100, (mean + 2 * std) * 100))

Accuracy with stratified dummy, 10 = [58.00, 58.00]


In [4]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier

SEED = 301
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, bad_x, bad_y, cv = 10, return_train_score=False)
mean = results['test_score'].mean()
std = results['test_score'].std()
print("Accuracy with cross validation, 10 = [%.2f, %.2f]" % ((mean - 2 * std)*100, (mean + 2 * std) * 100))

Accuracy with cross validation, 10 = [73.83, 77.73]


In [5]:
# Creating a "model" column to group cars by their model

np.random.seed(SEED)
data['model'] = data['models_age'] + np.random.randint(-2, 3, size=10000)
data['model'] = data['model'] + abs(data['model'].min()) + 1
data.head()

,price,sold,models_age,km_per_year,model
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


In [6]:
def print_results(results):
  mean = results['test_score'].mean() * 100
  std = results['test_score'].std() * 100
  print("Mean accuracy %.2f" % mean)
  print("Interval [%.2f, %.2f]" % (mean - 2 * std, mean + 2 * std))

In [7]:
# GroupKFold to analyse how the model behaves with new groups

from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, bad_x, bad_y, cv = cv, groups = data['model'], return_train_score=False)
print_results(results)

Mean accuracy 75.78
Interval [73.67, 77.90]


In [8]:
# GroupKFold in a pipeline with StandardScaler and SVC

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
model = SVC()

pipeline = Pipeline([('Scaler',scaler), ('Estimator',model)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, bad_x, bad_y, cv = cv, groups = data.model, return_train_score=False)
print_results(results)

Mean accuracy 76.68
Interval [74.28, 79.08]
